In [1]:
from node import *
import pulp as plp
import numpy as np
import pandas as pd

In [2]:
d=node('d',[])
b=node('b',[d])
c=node('c',[])
a=node('a',[b,c]);

In [3]:
f=node('f',[])
g=node('g',[])
e=node('e',[f,g])

In [4]:
def find_path_root(now,dfs,path,dfs_node,f):
    now.path=path.copy();
    now.f=f
    now.dfs=dfs;
    path.append(now);
    dfs_node.append(now);
    for i in now.son:
        dfs=find_path_root(i,dfs+1,path,dfs_node,now);
        
    path.remove(now)
    now.num_son = dfs-now.dfs;
    return dfs
n = 4
m = 3
set_I = range(0, n)
set_J = range(0, m)
c = {(i,j): 1 for i in set_I for j in set_J}


In [5]:
dfs_node1=[]
dfs_node2=[]
find_path_root(a,0,[],dfs_node1,a)
find_path_root(e,0,[],dfs_node2,e)

2

In [6]:
x_vars  = {(i,j):plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}".format(i,j)) for i in set_I for j in set_J}


In [7]:
opt_model = plp.LpProblem(name="MIP Model")
for i in set_I:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for j in set_J),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintI{0}".format(i)))

for j in set_J:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for i in set_I),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintJ{0}".format(j))) 
for i in dfs_node1:
    for j in dfs_node2:
        son_i = [dfs_node1[k] for k in range(i.dfs,i.dfs+i.num_son)];
        son_j = [dfs_node2[k] for k in range(j.dfs,j.dfs+j.num_son)];
        for k in i.path:
            for l in j.path:
                if(k==[]or l==[]):
                    continue;
                # print(i,j,k,l)
                opt_model.addConstraint(
                plp.LpConstraint(e=x_vars[i.dfs,j.dfs]+x_vars[i.dfs,l.dfs]+x_vars[k.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=2,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
for i in dfs_node1:
    if(len(i.son)==2):
        l=i.son[0];
        r=i.son[1];
        for j in dfs_node2:
            opt_model.addConstraint(
            plp.LpConstraint(e=x_vars[l.dfs,j.dfs]+x_vars[r.dfs,j.dfs]-x_vars[i.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=1,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
                

objective = plp.lpSum(x_vars[i,j] * c[i,j] for i in set_I for j in set_J)
opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)
opt_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/bcdd7fb13d4a4ba18e1459bd6ac4a312-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/bcdd7fb13d4a4ba18e1459bd6ac4a312-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 117 RHS
At line 136 BOUNDS
At line 149 ENDATA
Problem MODEL has 18 rows, 12 columns and 57 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9 - 0.00 seconds
Cgl0004I processed model has 15 rows, 11 columns (11 integer (11 of which binary)) and 47 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 6 integers unsatisfied sum - 3
Cbc0038I Solution found of -6
Cbc0038I Before mini bran

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [8]:
print('SOLUTION:')
for v in opt_model.variables():
    print(f'\t\t{v.name} = {v.varValue}')

print('\n') # Prints a blank line
print(f'OBJECTIVE VALUE: {opt_model.objective.value()}')

SOLUTION:
		x_0_0 = 1.0
		x_0_1 = 1.0
		x_0_2 = 1.0
		x_1_0 = 0.0
		x_1_1 = 1.0
		x_1_2 = 1.0
		x_2_0 = 0.0
		x_2_1 = 1.0
		x_2_2 = 1.0
		x_3_0 = 0.0
		x_3_1 = 1.0
		x_3_2 = 1.0


OBJECTIVE VALUE: 9.0


In [59]:
result_node = []
for v in opt_model.variables():
    if(v.value()==0):
        continue
    l = int(v.name.split('_')[1])
    r = int(v.name.split('_')[2])
    tn = newnode(dfs_node1[l],dfs_node2[r])
    result_node.append(tn);
    
for i in range(len(result_node)):
    for j in range(len(result_node)):
        if(i==j):
            continue
        p1 = result_node[i]
        p2 = result_node[j]
        if((p1.node1 in p2.node1.path or p1.node1 == p2.node1) and (p1.node2 in p2.node2.path or p1.node2==p2.node2)):
            result_node[i].edge.append(result_node[j])
            result_node[j].indegree +=1

def find_indgee(lists,indegree):
    ans=[]
    for i in lists:
        if(i.indegree == indegree):
            ans.append(i);
    return ans;
root=find_indgee(result_node,0)[0]
root

a_e

In [65]:
vis = result_node.copy()
nbs = find_indgee(vis,0)
vis.remove(nbs[0]);
while(len(nbs)!=0):
    node = nbs[0]
    nbs.pop(0)
    sons = find_indgee(node.edge,1);
    print(node,vis)
    for i in sons:
        i.f = node;
    for i in node.edge:
        i.indegree = i.indegree - 1;
    rs = find_indgee(vis,0)
    
    print(rs)
    for i in rs:
        vis.remove(i)
    nbs.extend(rs)


a_e [a_f, a_g, b_f, b_g, d_f, d_g, c_f, c_g]
[a_f, a_g]
a_f [b_f, b_g, d_f, d_g, c_f, c_g]
[b_f, c_f]
a_g [b_g, d_f, d_g, c_g]
[b_g, c_g]
b_f [d_f, d_g]
[d_f]
c_f [d_g]
[]
b_g [d_g]
[d_g]
c_g []
[]
d_f []
[]
d_g []
[]


In [67]:
for i in result_node:
    print(i,i.f)

a_e None
a_f a_e
a_g a_e
b_f a_f
b_g a_g
d_f b_f
d_g b_g
c_f a_f
c_g a_g


In [11]:
opt_model.variables()[0].varValue

1.0

In [5]:
c0=node('b7',[])
d0=node('b8',[])
f0=node('b3',[])
h0=node('b5',[])
b0=node('b6',[c0,d0])
g0=node('b4',[h0])
e0=node('b2',[f0,g0])
a0=node('b1',[b0,e0])



In [6]:
f1=node('r9',[])
g1=node('r10',[])
d1=node('r7',[])
e1=node('r8',[f1,g1])
c1=node('r6',[d1,e1])
b1=node('r5',[c1])
j1=node('r4',[])
i1=node('r3',[j1])
h1=node('r2',[i1])
a1=node('r1',[b1,h1])

In [18]:
def find_path_root(now,dfs,path,dfs_node,f):
    now.path=path.copy();
    now.f=f
    now.dfs=dfs;
    path.append(now);
    dfs_node.append(now);
    for i in now.son:
        dfs=find_path_root(i,dfs+1,path,dfs_node,now);
        
    path.remove(now)
    now.num_son = dfs-now.dfs;
    return dfs
n = 8
m = 10
set_I = range(0, n)
set_J = range(0, m)
c = {(i,j): 0 for i in set_I for j in set_J}


In [24]:
temp = [(1,1),(5,8),(6,8),(7,9),(8,9),(7,10),(1,2),(1,3),(2,2),(3,2),(4,2),(1,4),(1,5),(1,6),(1,7)]
for i,j in temp:
    c[(i-1,j-1)]=10;

In [25]:
dfs_node1=[]
dfs_node2=[]
find_path_root(a0,0,[],dfs_node1,a0)
find_path_root(a1,0,[],dfs_node2,a1)
x_vars  = {(i,j):plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}".format(dfs_node1[i],dfs_node2[j])) for i in set_I for j in set_J}


In [26]:
opt_model = plp.LpProblem(name="MIP Model")
for i in set_I:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for j in set_J),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintI{0}".format(i)))

for j in set_J:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for i in set_I),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintJ{0}".format(j))) 
for i in dfs_node1:
    for j in dfs_node2:
        son_i = [dfs_node1[k] for k in range(i.dfs,i.dfs+i.num_son)];
        son_j = [dfs_node2[k] for k in range(j.dfs,j.dfs+j.num_son)];
        for k in i.path:
            for l in j.path:
                if(k==[]or l==[]):
                    continue;
                # print(i,j,k,l)
                opt_model.addConstraint(
                plp.LpConstraint(e=x_vars[i.dfs,j.dfs]+x_vars[i.dfs,l.dfs]+x_vars[k.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=2,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
for i in dfs_node1:
    if(len(i.son)==2):
        l=i.son[0];
        r=i.son[1];
        for j in dfs_node2:
            opt_model.addConstraint(
            plp.LpConstraint(e=x_vars[l.dfs,j.dfs]+x_vars[r.dfs,j.dfs]-x_vars[i.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=1,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
                

objective = plp.lpSum(x_vars[i,j] * c[i,j] for i in set_I for j in set_J)
opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)
opt_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/c4e6385c4f6144ada8bb7fd58c48d0e8-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/c4e6385c4f6144ada8bb7fd58c48d0e8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 352 COLUMNS
At line 1676 RHS
At line 2024 BOUNDS
At line 2105 ENDATA
Problem MODEL has 347 rows, 80 columns and 1147 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 150 - 0.00 seconds
Cgl0004I processed model has 344 rows, 79 columns (79 integer (79 of which binary)) and 1126 elements
Cutoff increment increased from 1e-05 to 9.9999
Cbc0045I 15 integer variables out of 79 objects (79 integer) have cost of -10 - high priority
Cbc00

1

In [27]:
print('SOLUTION:')
for v in opt_model.variables():
    print(f'\t\t{v.name} = {v.varValue}')

print('\n') # Prints a blank line
print(f'OBJECTIVE VALUE: {opt_model.objective.value()}')

SOLUTION:
		x_b1_r1 = 1.0
		x_b1_r10 = 1.0
		x_b1_r2 = 0.0
		x_b1_r3 = 0.0
		x_b1_r4 = 0.0
		x_b1_r5 = 1.0
		x_b1_r6 = 1.0
		x_b1_r7 = 1.0
		x_b1_r8 = 1.0
		x_b1_r9 = 1.0
		x_b2_r1 = 0.0
		x_b2_r10 = 0.0
		x_b2_r2 = 1.0
		x_b2_r3 = 0.0
		x_b2_r4 = 0.0
		x_b2_r5 = 0.0
		x_b2_r6 = 0.0
		x_b2_r7 = 0.0
		x_b2_r8 = 0.0
		x_b2_r9 = 0.0
		x_b3_r1 = 0.0
		x_b3_r10 = 0.0
		x_b3_r2 = 1.0
		x_b3_r3 = 0.0
		x_b3_r4 = 0.0
		x_b3_r5 = 0.0
		x_b3_r6 = 0.0
		x_b3_r7 = 0.0
		x_b3_r8 = 0.0
		x_b3_r9 = 0.0
		x_b4_r1 = 0.0
		x_b4_r10 = 0.0
		x_b4_r2 = 0.0
		x_b4_r3 = 1.0
		x_b4_r4 = 1.0
		x_b4_r5 = 0.0
		x_b4_r6 = 0.0
		x_b4_r7 = 0.0
		x_b4_r8 = 0.0
		x_b4_r9 = 0.0
		x_b5_r1 = 0.0
		x_b5_r10 = 0.0
		x_b5_r2 = 0.0
		x_b5_r3 = 1.0
		x_b5_r4 = 0.0
		x_b5_r5 = 0.0
		x_b5_r6 = 0.0
		x_b5_r7 = 0.0
		x_b5_r8 = 0.0
		x_b5_r9 = 0.0
		x_b6_r1 = 0.0
		x_b6_r10 = 0.0
		x_b6_r2 = 0.0
		x_b6_r3 = 0.0
		x_b6_r4 = 0.0
		x_b6_r5 = 1.0
		x_b6_r6 = 0.0
		x_b6_r7 = 0.0
		x_b6_r8 = 0.0
		x_b6_r9 = 0.0
		x_b7_r1 = 0.0
		x_b7_r